In [ ]:
!pip install langdetect
!pip install pycountry

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 48.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import json
from langdetect import detect, DetectorFactory, LangDetectException
import pycountry
import json
import re


DetectorFactory.seed = 0  # For consistent results

In [ ]:

# Read the CSV file
df = pd.read_csv('/content/Instagram-datasets.csv')


In [ ]:

# Display basic information
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   url                      1000 non-null   object
 1   comment_user             1000 non-null   object
 2   comment_user_url         1000 non-null   object
 3   comment_date             1000 non-null   object
 4   comment                  1000 non-null   object
 5   likes_number             1000 non-null   int64 
 6   replies_number           1000 non-null   int64 
 7   replies                  998 non-null    object
 8   hashtag_comment          6 non-null      object
 9   tagged_users_in_comment  146 non-null    object
 10  post_url                 1000 non-null   object
 11  post_user                1000 non-null   object
 12  comment_id               1000 non-null   int64 
 13  post_id                  1000 non-null   int64 
dtypes: int64(4), object(10)
memory usage: 109

In [ ]:
print(df.head(5))

                                          url        comment_user  \
0          https://www.instagram.com/badespi_            fab███vf   
1          https://www.instagram.com/badespi_        des███opi███   
2        https://www.instagram.com/cojuvpiaui        mar███osb███   
3        https://www.instagram.com/gamilpiaui        tic███cau███   
4  https://www.instagram.com/agespisa.oficial  bru███nca███cao███   

                            comment_user_url              comment_date  \
0         https://www.instagram.com/fabiacvf  2024-11-13T20:01:57.000Z   
1     https://www.instagram.com/destinopiaui  2024-11-13T17:11:39.000Z   
2      https://www.instagram.com/marciaosb16  2024-11-13T23:00:46.000Z   
3      https://www.instagram.com/ticimacauff  2024-11-14T00:16:19.000Z   
4  https://www.instagram.com/brunoencarnacao  2024-11-14T17:33:11.000Z   

                                             comment  likes_number  \
0                                                👏👏👏             1   


In [ ]:
comments = df['comment'].to_list()

replies = df['replies'].to_list()

In [ ]:
print(comments)

['👏👏👏', '😍😍😍', 'Meu querido @euwanderson7  que  foi nosso delegado representando o nosso estado e o Vale do Sambito, Novo Oriente do Piauí.', 'Com o professor Matheus Carvalho, realmente  sensacional 👏👏👏👏', '@rafael.fonteles todo ano é isso. O único município na América do Sul, que fica sem água quando a água chega. Os cidadãos já não aguentam mais. Vai resolver isso quando? Delega alguém pra RESOLVER esse problema, meu camarada. Não existe isso, com orçamento que tem, 6 dias sem água. Vocês não resolvem pq não querem.', '👏❤️', '👏👏👏', 'Ovinos, bubalinos , bovinos, equinos', 'O cadastro é grátis ou paga taxa?', 'Orgulho dessa arquiteta genial que está a frente desse projeto de construção! @marinabsant', 'Ficou muito linda, e a inauguração foi emocionante!👏👏👏', 'Foi enriquecedor 👏🏻👏🏻👏🏻', '👍👍👏👏', 'Pura propaganda mentirosa, tem médico sendo bloqueado pq pede exames para o paciente, aí fica 45 dias o médico bloqueado e o segurado sem atendimento, uma das piores gestão.', 'Onde posso tira p

In [ ]:
print(replies)

['[{"likes_number":1,"reply":"@fabiacvf 👏👏👏","reply_date":"2024-11-14T12:38:40.000Z","reply_user":"bad███i_"}]', '[{"likes_number":0,"reply":"@destinopiaui 😍👏👏","reply_date":"2024-11-13T19:08:22.000Z","reply_user":"bad███i_"}]', '[{"likes_number":0,"reply":"@marciaosb16 ❤️🫶🥰","reply_date":"2024-11-13T23:31:59.000Z","reply_user":"euw███ers███"}]', '[{"likes_number":0,"reply":"👏👏🤝🤝","reply_date":"2024-11-14T02:54:24.000Z","reply_user":"esc███deg███rno█████████"}]', '[{"likes_number":1,"reply":"Nós não merecemos isso! Corrente não merece isso!","reply_date":"2024-11-14T17:33:38.000Z","reply_user":"bru███nca███cao███"}]', '[{"likes_number":0,"reply":"@flordosertaoturismo obrigado 🙏","reply_date":"2024-11-13T22:58:07.000Z","reply_user":"zen███mon███roo███"}]', '[{"likes_number":1,"reply":"@joao_angelinejunior vamo junto amigo 👏","reply_date":"2024-11-13T22:58:13.000Z","reply_user":"zen███mon███roo███"}]', '[{"likes_number":1,"reply":"@rodriguesraimunda_2005 👏","reply_date":"2024-11-08T17:54

In [ ]:

all_replies = []

for reply_list in replies:
    if isinstance(reply_list, str):  # Only process if it's a string
        try:
            parsed = json.loads(reply_list)
            # print(len(parsed),end=" ")
            # print(parsed,end="\n")
            for reply in parsed:
                all_replies.append(reply.get('reply', ''))
        except json.JSONDecodeError:
            continue



1 [{'likes_number': 1, 'reply': '@fabiacvf 👏👏👏', 'reply_date': '2024-11-14T12:38:40.000Z', 'reply_user': 'bad███i_'}]
1 [{'likes_number': 0, 'reply': '@destinopiaui 😍👏👏', 'reply_date': '2024-11-13T19:08:22.000Z', 'reply_user': 'bad███i_'}]
1 [{'likes_number': 0, 'reply': '@marciaosb16 ❤️🫶🥰', 'reply_date': '2024-11-13T23:31:59.000Z', 'reply_user': 'euw███ers███'}]
1 [{'likes_number': 0, 'reply': '👏👏🤝🤝', 'reply_date': '2024-11-14T02:54:24.000Z', 'reply_user': 'esc███deg███rno█████████'}]
1 [{'likes_number': 1, 'reply': 'Nós não merecemos isso! Corrente não merece isso!', 'reply_date': '2024-11-14T17:33:38.000Z', 'reply_user': 'bru███nca███cao███'}]
1 [{'likes_number': 0, 'reply': '@flordosertaoturismo obrigado 🙏', 'reply_date': '2024-11-13T22:58:07.000Z', 'reply_user': 'zen███mon███roo███'}]
1 [{'likes_number': 1, 'reply': '@joao_angelinejunior vamo junto amigo 👏', 'reply_date': '2024-11-13T22:58:13.000Z', 'reply_user': 'zen███mon███roo███'}]
1 [{'likes_number': 1, 'reply': '@rodriguesra

In [ ]:
# Display replies
for r in all_replies:
    print(r)

@fabiacvf 👏👏👏
@destinopiaui 😍👏👏
@marciaosb16 ❤️🫶🥰
👏👏🤝🤝
Nós não merecemos isso! Corrente não merece isso!
@flordosertaoturismo obrigado 🙏
@joao_angelinejunior vamo junto amigo 👏
@rodriguesraimunda_2005 👏
@juvenciolima58 boa noite Juvêncio. Tanto o cadastro como  a atualização que começou dia 1/11 e vai até 21/12 são serviços sem custos ao produtor.
@adapi_pi muito obrigado
@drajessicanneves 😍
@karla.aandrade  tudo muito lindo e muito emocionante mesmo !!! Digno da Defensoria !!!❤️❤️
@ednaelma_lima 👏👏
@valdeneclementino 👏👏
@marvini.oli  pois é! Eu não sei onde são feitos esses atendimentos, pq tentei neuro e endócrino para minha filha e não consegui.  Tentei endócrino, pra mim e estava suspenso. Tentei cardio  ao tinha vaga e ginecologista, agendei em setembro somente para novembro.  Queria era saber onde são essas clínicas , pq acho que estou me direcionando para as erradas.  Sem falar no site, que está totalmente desatualizado.
@regina_marcia_b vamos te passar os documentos no direct
@

In [ ]:
language_counts_comments = {}
hashed_comm = []
for text in comments:
    if "#" in text :
        hashed_comm.append(text)
    try:
        lang = detect(text)
    except LangDetectException:
        lang = "unknown"
    language_counts_comments[lang] = language_counts_comments.get(lang, 0) + 1



In [ ]:

print(language_counts_comments)

print(hashed_comm)

{'unknown': 179, 'pt': 79, 'lt': 4, 'ca': 6, 'en': 343, 'es': 22, 'hr': 5, 'fr': 24, 'ro': 20, 'tl': 11, 'sl': 5, 'it': 17, 'ar': 133, 'de': 10, 'he': 6, 'af': 14, 'nl': 16, 'pl': 10, 'id': 13, 'fi': 10, 'th': 1, 'sw': 10, 'tr': 3, 'fa': 12, 'cy': 6, 'so': 10, 'vi': 4, 'ur': 1, 'et': 6, 'mr': 1, 'ko': 1, 'hu': 1, 'no': 7, 'sv': 5, 'ml': 1, 'sk': 2, 'da': 1, 'el': 1}
['@tv3_ghana You guys have been fantastic with the election coverage!  God bless you for sacrificing your time and energy out there. \nWe appreciate and love you all. We love TV3!\n #First in News #Best In Entertainment', '#deluded', 'Can’t wait to join forces with @linkedinexpert & @jonferrara to share #Sales tips using content.', 'Oto Patamar! A expansão, melhoria do ensino médio em todo o estado é uma #ConquistaDasJuventudes desde as conferências e garantidas pelo Plano Estadual da Juventude Piauiense. #EstatutoDaJuventude', '#marketing #sales #socialselling #socialsales #smallbusiness #smallbiz #SMB #CRM #socialCRM #pro

In [ ]:
import re

# Initialize the dictionary to store sentences with hashtags and the phrases after hashtags
hashtag_sentences = {}

# Function to extract full hashtag phrases (the text following each hashtag)
def extract_hashtag_phrases(text):
    # This regex captures a hashtag and the words that follow it until the next hashtag or end of the sentence
    return re.findall(r'#(\w+(?: \w+)*)', text)

# Loop through the comments and process them
for text in comments:
    if "#" in text:  # Check if the comment contains hashtags
        hashtag_phrases = extract_hashtag_phrases(text)  # Extract phrases after hashtags
        hashtag_sentences[text] = hashtag_phrases  # Store the sentence and its corresponding hashtag phrases in the dictionary


for text in all_replies:
    if "#" in text:  # Check if the comment contains hashtags
        hashtag_phrases = extract_hashtag_phrases(text)  # Extract phrases after hashtags
        hashtag_sentences[text] = hashtag_phrases  # Store the sentence and its corresponding hashtag phrases in the dictionary



In [ ]:
# Print the resulting dictionary
for sentence, phrases in hashtag_sentences.items():
    print(f"Sentence: {sentence}")
    print(f"Hashtag Phrases: {phrases}\n")

Sentence: @tv3_ghana You guys have been fantastic with the election coverage!  God bless you for sacrificing your time and energy out there. 
We appreciate and love you all. We love TV3!
 #First in News #Best In Entertainment
Hashtag Phrases: ['First in News', 'Best In Entertainment']

Sentence: #deluded
Hashtag Phrases: ['deluded']

Sentence: Can’t wait to join forces with @linkedinexpert & @jonferrara to share #Sales tips using content.
Hashtag Phrases: ['Sales tips using content']

Sentence: Oto Patamar! A expansão, melhoria do ensino médio em todo o estado é uma #ConquistaDasJuventudes desde as conferências e garantidas pelo Plano Estadual da Juventude Piauiense. #EstatutoDaJuventude
Hashtag Phrases: ['ConquistaDasJuventudes desde as conferências e garantidas pelo Plano Estadual da Juventude Piauiense', 'EstatutoDaJuventude']

Sentence: #marketing #sales #socialselling #socialsales #smallbusiness #smallbiz #SMB #CRM #socialCRM #prospecting #startup #startups #contactmanagement #bus

In [ ]:
hashed_commnets_df = df['hashtag_comment'].to_list()


cleaned_hashtags = [text for text in hashed_commnets_df if text and not pd.isna(text)]

# Print the cleaned list
print(cleaned_hashtags)

['["#First","#Best"]', '["#deluded"]', '["#Sales"]', '["#ConquistaDasJuventudes","#EstatutoDaJuventude"]', '["#marketing","#sales","#socialselling","#socialsales","#smallbusiness","#smallbiz","#SMB","#CRM","#socialCRM","#prospecting","#startup","#startups","#contactmanagement","#business","#SaaS","#Gsuite","#O365","#Officece365","#Microsoft","#socialmedia","#entrepreneur","#entrepreneurship","#influencer","#influencermarketing"]', '["#crm","#solopreneur"]']


In [ ]:
# Convert codes to full names
language_counts_comments_full = {}

for code, count in language_counts_comments.items():
    if code == "unknown":
        name = "Unknown"
    else:
        language = pycountry.languages.get(alpha_2=code)
        name = language.name if language else f"Unknown ({code})"
    language_counts_comments_full[name] = count

# Print result
for lang, count in language_counts_comments_full.items():
    print(f"{lang}: {count}")

Unknown: 179
Portuguese: 79
Lithuanian: 4
Catalan: 6
English: 343
Spanish: 22
Croatian: 5
French: 24
Romanian: 20
Tagalog: 11
Slovenian: 5
Italian: 17
Arabic: 133
German: 10
Hebrew: 6
Afrikaans: 14
Dutch: 16
Polish: 10
Indonesian: 13
Finnish: 10
Thai: 1
Swahili (macrolanguage): 10
Turkish: 3
Persian: 12
Welsh: 6
Somali: 10
Vietnamese: 4
Urdu: 1
Estonian: 6
Marathi: 1
Korean: 1
Hungarian: 1
Norwegian: 7
Swedish: 5
Malayalam: 1
Slovak: 2
Danish: 1
Modern Greek (1453-): 1


In [ ]:

# Detect languages in all_replies
language_counts_replies = {}
replies_with_lang = []

for reply in all_replies:
    try:
        lang = detect(reply)
    except LangDetectException:
        lang = "unknown"
    language_counts_replies[lang] = language_counts_replies.get(lang, 0) + 1
    replies_with_lang.append((reply, lang))

# Convert codes to full names for replies
language_counts_replies_full = {}
for code, count in language_counts_replies.items():
    if code == "unknown":
        name = "Unknown"
    else:
        language = pycountry.languages.get(alpha_2=code)
        name = language.name if language else f"Unknown ({code})"
    language_counts_replies_full[name] = count





In [ ]:
# Print reply language distribution
print("Reply Language Counts:")
for lang, count in language_counts_replies_full.items():
    print(f"{lang}: {count}")

Reply Language Counts:
Italian: 36
Romanian: 15
Portuguese: 81
Unknown: 11
Spanish: 38
Lithuanian: 20
French: 28
Estonian: 35
Dutch: 26
Catalan: 15
Hungarian: 11
English: 373
Swahili (macrolanguage): 24
Tagalog: 24
Indonesian: 34
Finnish: 30
Norwegian: 13
Afrikaans: 24
Swedish: 9
Somali: 107
Slovenian: 9
Hebrew: 7
Welsh: 24
Latvian: 5
Arabic: 62
Turkish: 16
Albanian: 7
Croatian: 7
Danish: 9
Polish: 9
German: 12
Slovak: 7
Vietnamese: 3
Modern Greek (1453-): 1


In [ ]:
# Combine all texts from comments and replies with language info
all_texts = []

# Add comments with language
for text in comments:
    try:
        lang = detect(text)
    except LangDetectException:
        lang = "unknown"
    language = pycountry.languages.get(alpha_2=lang)
    lang_name = language.name if language else "Unknown" if lang == "unknown" else f"Unknown ({lang})"
    all_texts.append({"text": text, "language_code": lang, "language_name": lang_name})

# Add replies with language (already detected above)
for reply, lang in replies_with_lang:
    language = pycountry.languages.get(alpha_2=lang)
    lang_name = language.name if language else "Unknown" if lang == "unknown" else f"Unknown ({lang})"
    all_texts.append({"text": reply, "language_code": lang, "language_name": lang_name})




In [ ]:
# # Display sample of combined results
# print("\nCombined Texts with Language:")
# for entry in all_texts[:10]:  # show first 10
#     print(f"[{entry['language_name']}] {entry['text']}")

In [ ]:
# Initialize a dictionary for language counts
language_counts_total = {}

# Helper function to get full name from code
def get_language_name(code):
    if code == "unknown":
        return "Unknown"
    lang = pycountry.languages.get(alpha_2=code)
    return lang.name if lang else f"Unknown ({code})"

# Detect languages in comments
for text in comments:
    try:
        lang = detect(text)
    except LangDetectException:
        lang = "unknown"
    lang_name = get_language_name(lang)
    language_counts_total[lang_name] = language_counts_total.get(lang_name, 0) + 1

# Detect languages in all_replies
for text in all_replies:
    try:
        lang = detect(text)
    except LangDetectException:
        lang = "unknown"
    lang_name = get_language_name(lang)
    language_counts_total[lang_name] = language_counts_total.get(lang_name, 0) + 1



In [ ]:
# Print final combined result
print("Final Total Language Mapping (Comments + Replies):")
for lang_name, count in sorted(language_counts_total.items(), key=lambda x: x[1], reverse=True):
    print(f"{lang_name}: {count}")

Final Total Language Mapping (Comments + Replies):
English: 716
Arabic: 195
Unknown: 190
Portuguese: 160
Somali: 117
Spanish: 60
Italian: 53
French: 52
Indonesian: 47
Dutch: 42
Estonian: 41
Finnish: 40
Afrikaans: 38
Romanian: 35
Tagalog: 35
Swahili (macrolanguage): 34
Welsh: 30
Lithuanian: 24
German: 22
Catalan: 21
Norwegian: 20
Polish: 19
Turkish: 19
Slovenian: 14
Swedish: 14
Hebrew: 13
Croatian: 12
Persian: 12
Hungarian: 12
Danish: 10
Slovak: 9
Vietnamese: 7
Albanian: 7
Latvian: 5
Modern Greek (1453-): 2
Thai: 1
Urdu: 1
Marathi: 1
Korean: 1
Malayalam: 1


In [ ]:
# Print final combined result
print("Final Total Language Mapping (Comments + Replies):")
for lang_name, count in sorted(language_counts_total.items(), key=lambda x: x[0]):
    print(f"{lang_name}: {count}")

Final Total Language Mapping (Comments + Replies):
Afrikaans: 38
Albanian: 7
Arabic: 195
Catalan: 21
Croatian: 12
Danish: 10
Dutch: 42
English: 716
Estonian: 41
Finnish: 40
French: 52
German: 22
Hebrew: 13
Hungarian: 12
Indonesian: 47
Italian: 53
Korean: 1
Latvian: 5
Lithuanian: 24
Malayalam: 1
Marathi: 1
Modern Greek (1453-): 2
Norwegian: 20
Persian: 12
Polish: 19
Portuguese: 160
Romanian: 35
Slovak: 9
Slovenian: 14
Somali: 117
Spanish: 60
Swahili (macrolanguage): 34
Swedish: 14
Tagalog: 35
Thai: 1
Turkish: 19
Unknown: 190
Urdu: 1
Vietnamese: 7
Welsh: 30


In [ ]:
twitter_df = pd.read_csv('/content/Twitter- datasets.csv')

In [ ]:
print(twitter_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 26 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   1000 non-null   int64 
 1   user_posted          1000 non-null   object
 2   name                 1000 non-null   object
 3   description          1000 non-null   object
 4   date_posted          1000 non-null   object
 5   photos               456 non-null    object
 6   url                  1000 non-null   object
 7   tagged_users         374 non-null    object
 8   replies              1000 non-null   int64 
 9   reposts              1000 non-null   int64 
 10  likes                1000 non-null   int64 
 11  views                1000 non-null   int64 
 12  external_url         432 non-null    object
 13  hashtags             572 non-null    object
 14  followers            1000 non-null   int64 
 15  biography            997 non-null    object
 16  posts_c

In [ ]:
# Assuming 'df' is your DataFrame
for column in twitter_df.columns:
    print(f"Column: {column}")
    print(twitter_df[column].iloc[:25].to_list())  # Print the first 5 rows for this column
    print("\n" + "-"*50 + "\n")


Column: id
[1868428607451799983, 1868159094567121215, 1868451534708883739, 1868441382022717466, 1868418260892565925, 1868380932962418758, 1868386052634784163, 1868362011253301608, 1868453532929552384, 1868430929389105388, 1868083722471235914, 1868121373425516847, 1868407672376058098, 1868383589420707992, 1868352193499197494, 1868113795878461891, 1868463142856180087, 1868421338333077960, 1868429967177957452, 1868401245876085034, 1868468740171673945, 1868453542400254164, 1868445939360321723, 1868418200008012245, 1868415158818603228]

--------------------------------------------------

Column: user_posted
['Glo███ews███', 'bil███ard███', 'TNT███rts███', 'TNT███rts███', 'Glo███ews███', 'Glo███ews███', 'bil███ard███', 'Glo███ews███', 'bil███ard███', 'bil███ard███', 'bil███ard███', 'bil███ard███', 'Glo███ews███', 'Glo███ews███', 'Glo███ews███', 'bil███ard███', 'TNT███rts███', 'TNT███rts███', 'Glo███ews███', 'Glo███ews███', 'TNT███rts███', 'TNT███rts███', 'TNT███rts███', 'Glo███ews███', 'Glo█